# Объединение выгруженных csv-файлов сделок в единый датафрейм 
- Принимает файл/файлы, выгруженные с [сайта finam](https://www.finam.ru/quote/moex/gazp/export/) (в качестве примера - котировки Газпрома) с настройками:   
    - интервал: тики,   
    - формат: csv,   
    - формат даты: ггггммдд,  
    - формат времени: ччммсс,  
    - начало свечи:  московское,   
    - разделитель полей: запятая,  
    - разрядов: нет,    
    - формат записи в файл: date-time-last-vol-id-oper.  
- Записывает данные в файл quotes.csv
- Убеждается в отсутствии дубликатов  
- Удаляет использованные файлы
- Формат выходного файла: full_date, last, volume, id, direction, delta_volume, delta_trades


### Импорты/настройки

In [1]:
import pandas as pd
import os
import warnings

warnings.filterwarnings('ignore') 

DIR = 'Datasets/donor/' # Директория, откуда загружаются файлы
FILE_LIST = os.listdir('Datasets/donor') # Список файлов для обработки
PATH_OUT = 'Datasets/data.csv' # Выходной файл

def memo_reductio(data):
    ''' 
    Уменьшает размерность данных для ускорения обработки
    '''
    data['full_date'] = pd.to_datetime(data['full_date'], format='%Y-%m-%d %H:%M:%S')
    data['direction'] = data['direction'].astype('category')
    data[['volume','id']] = data[['volume','id']].astype('uint64')
    return data

def delta_volume(volume, direction):
    '''
    Создание дельты по объему
    '''
    return volume if direction == 'B' else -volume

def delta_trades(direction):
    '''
    Создание дельты по сделкам
    '''
    return -1 if direction == 'S' else 1 

def processing(data):
    '''
    Упаковка функций для уменьшения кода
    '''
    data = data.rename(columns={'<DATE>_<TIME>':'full_date','<LAST>':'last', '<VOL>':'volume', '<ID>':'id','<OPER>':'direction'})
    #data = memo_reductio(data) 
    data['delta_volume'] = data.apply(lambda row: delta_volume(row['volume'], row['direction']), axis=1).astype('int32')
    data['delta_trades'] = data['direction'].apply(delta_trades).astype('int32')
    data = data.drop('direction', axis=1)
    return data

### Обработка

In [2]:
# Проверка, есть ли уже файл с котировками
try:
    quotes = pd.read_csv('Datasets/quotes.csv', parse_dates=['full_date'] ) 
    quotes[['volume','id']] = quotes[['volume','id']].astype('uint64')
    print('file exists')
    
# Если файла с котировками нет, создает новый
except: 
    quotes = pd.read_csv(DIR+FILE_LIST[0], parse_dates=[['<DATE>', '<TIME>']])
    quotes = processing(quotes)
    print('creating new file')

file exists


In [3]:
%%time
for i in FILE_LIST:
    PATH_IN = DIR + i
    data = pd.read_csv(PATH_IN, parse_dates=[['<DATE>', '<TIME>']])
    data = processing(data)
    
    print(f'Обработка файла {i}, {len(data)} сделок')
    quotes = pd.concat([quotes, data])
    print(f'Удалено дубликатов: {quotes.duplicated().sum()}')
    quotes = quotes.drop_duplicates(keep='first')
    os.remove(DIR + i)
    print(f'Обработано\n') 
    
print(f'Длина датасета: {len(quotes)}')
print(f'Дубликатов в датасете: {quotes.duplicated().sum()}\n')
quotes.sort_values(by='id', ignore_index=True, inplace=True)
quotes.to_csv('Datasets/quotes.csv', index=False)
print('Done!')

Обработка файла GAZP_230401_230409.csv, 419486 сделок
Удалено дубликатов: 419486
Обработано

Обработка файла GAZP_230410_230416.csv, 489725 сделок
Удалено дубликатов: 489725
Обработано

Обработка файла GAZP_230417_230423.csv, 357417 сделок
Удалено дубликатов: 357417
Обработано

Обработка файла GAZP_230424_230430.csv, 415816 сделок
Удалено дубликатов: 415816
Обработано

Обработка файла GAZP_230501_230514.csv, 685047 сделок
Удалено дубликатов: 685047
Обработано

Обработка файла GAZP_230515_230521.csv, 529947 сделок
Удалено дубликатов: 529947
Обработано

Обработка файла GAZP_230522_230528 (1).csv, 846659 сделок
Удалено дубликатов: 846659
Обработано

Обработка файла GAZP_230529_230531.csv, 294886 сделок
Удалено дубликатов: 294886
Обработано

Обработка файла GAZP_230601_230611.csv, 466859 сделок
Удалено дубликатов: 466859
Обработано

Обработка файла GAZP_230612_230618.csv, 287710 сделок
Удалено дубликатов: 287710
Обработано

Обработка файла GAZP_230619_230625.csv, 314568 сделок
Удалено дубл